# Assignment 3 - Transliteration

### Setup [do not change]

In [ ]:
!git clone https://github.com/butoialexandra/eth-nlp-f22-hw3.git

import os
os.chdir('eth-nlp-f22-hw3')

!pip install -e .

Cloning into 'eth-nlp-f22-hw3'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 29 (delta 6), reused 16 (delta 2), pack-reused 0
Unpacking objects: 100% (29/29), 9.90 KiB | 921.00 KiB/s, done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/eth-nlp-f22-hw3
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 KB 3.9 MB/s eta 0:00:00
  Running setup.py develop for rayuela


In [ ]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1wlHbFChZFcBgEU8knAeaUIqE0W4EaJEC' -O 'test_cases.pkl'

--2023-02-08 06:11:03--  https://docs.google.com/uc?export=download&id=1wlHbFChZFcBgEU8knAeaUIqE0W4EaJEC
Resolving docs.google.com (docs.google.com)... 74.125.197.100, 74.125.197.101, 74.125.197.102, ...
Connecting to docs.google.com (docs.google.com)|74.125.197.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0s-50-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/uc4dnvrd98ml9c97nv5v8gqb1t7mta78/1675836600000/03249710210786382974/*/1wlHbFChZFcBgEU8knAeaUIqE0W4EaJEC?e=download&uuid=ddda89ae-7f0f-4c6f-b05e-dda7dd5b6fc7 [following]
--2023-02-08 06:11:03--  https://doc-0s-50-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/uc4dnvrd98ml9c97nv5v8gqb1t7mta78/1675836600000/03249710210786382974/*/1wlHbFChZFcBgEU8knAeaUIqE0W4EaJEC?e=download&uuid=ddda89ae-7f0f-4c6f-b05e-dda7dd5b6fc7
Resolving doc-0s-50-docs.googleusercontent.com (doc-0s-50-docs.googleusercontent.com)... 74.125.20.132, 2607:f8b0:4

In [ ]:
from rayuela.base.semiring import Semiring, Real, Tropical
from rayuela.base.symbol import Sym, ε, ε_1, ε_2
from rayuela.base.misc import epsilon_filter

from rayuela.fsa.fst import FST
from rayuela.fsa.state import State, PairState

import dill
import numpy as np
from math import exp, log

from itertools import product

from frozendict import frozendict

### Subquestion a) - Log Semiring

In [ ]:
class Log(Semiring):
    def __init__(self, score):
        self.score = score

    def star(self):
        if self.score == 0:
          return None
        else:
          return Log(log(1.0 / (1.0 - exp(self.score))))

    def __float__(self):
        return float(self.score)

    def __add__(self, other):
        if self == self.zero and other == self.zero:
          return self.zero
        elif self == self.zero:
          return other
        elif other == self.zero :
          return self
        return Log(log(exp(self.score) + exp(other.score)))

    def __mul__(self, other):
        if other is self.one:
            return self
        if self is self.one:
            return other
        if other is self.zero:
            return self.zero
        if self is self.zero:
            return self.zero
        return Log(self.score + other.score)

    def __repr__(self):
        return f"{round(self.score, 15)}"

    def __eq__(self, other):
        return np.allclose(float(self.score), float(other.score), atol=1e-3)

    def __hash__(self):
        return hash(self.score)

Log.zero = Log(-float("inf"))
Log.one = Log(0)

Log.idempotent = False
Log.cancellative = True

### Before moving to the next subquestion, we give some examples of WFSTs to illustrate how to use the library `rayuela`.

In [ ]:
# A simple language model encoded by a WFST

sem = Real # the Real semiring
fst = FST(sem) # a WFST in the Real semiring

# add transitions
fst.add_arc(State(0), Sym('formal'), Sym('formal'), State(1), Real(0.2))
fst.add_arc(State(0), Sym('natural'), Sym('natural'), State(1), Real(0.3))
fst.add_arc(State(0), Sym('learning'), Sym('learning'), State(2), Real(0.2))
fst.add_arc(State(0), Sym('data'), Sym('data'), State(3), Real(0.3))

fst.add_arc(State(1), Sym('language'), Sym('language'), State(2), Real(0.6))
fst.add_arc(State(1), Sym('languages'), Sym('languages'), State(4), Real(0.4))

fst.add_arc(State(2), Sym('is'), Sym('is'), State(5), Real(1.0))

fst.add_arc(State(5), Sym('fun'), Sym('fun'), State(6), Real(1.0))

fst.add_arc(State(3), Sym('is'), Sym('is'), State(5), Real(0.5))
fst.add_arc(State(3), Sym('are'), Sym('are'), State(5), Real(0.5))

fst.add_arc(State(4), Sym('are'), Sym('are'), State(5), Real(1.0))

# set initial weights
# if not set, it defaults to semiring zero
fst.set_I(State(0), Real(1.0))
# set final weights
fst.set_F(State(6), Real(1.0))

# visualize
fst

WFSA(7 states, <class 'rayuela.base.semiring.Real'>)

In [ ]:
# A simple WFSA for transliteration

sem = Real # semiring
fst = FST(sem) # initialize WFST in the chosen semiring
one, zero = sem.one, sem.zero # semiring one and zero

# add transitions
fst.add_arc(State(0), Sym('d'), Sym('data'), State(1), Real(0.5))
fst.add_arc(State(0), Sym('d'), Sym('dew'), State(5), Real(0.5))

fst.add_arc(State(1), Sym('ey'), Sym('ÃŽÂµ'), State(2), Real(0.5))
fst.add_arc(State(1), Sym('ae'), Sym('ÃŽÂµ'), State(2), Real(0.5))

fst.add_arc(State(2), Sym('t'), Sym('ÃŽÂµ'), State(3), Real(0.7))
fst.add_arc(State(2), Sym('dx'), Sym('ÃŽÂµ'), State(3), Real(0.3))

fst.add_arc(State(3), Sym('ax'), Sym('ÃŽÂµ'), State(4), one)

fst.add_arc(State(5), Sym('uw'), Sym('ÃŽÂµ'), State(6), one)

# initial and final weights
fst.set_I(State(0), one)
fst.set_F(State(4), one)
fst.set_F(State(6), one)

fst

WFSA(7 states, <class 'rayuela.base.semiring.Real'>)

### Subquestion b) - String Transducer

In [ ]:
from numpy.core.numeric import ones
def string_fst(s, semiring=Log):
    """ Takes a string as input and a semiring
    and returns a transducer encoding of the
    string
    """
    fst = FST(semiring)
    one, zero = fst.R.one, fst.R.zero
    
    for i, el in enumerate(s):
        fst.add_arc(State(i), Sym(s[i]), Sym(s[i]), State(i+1), semiring(-10.0))

    # set initial weights
    # if not set, it defaults to semiring zero
    fst.set_I(State(0), semiring(-1.0))
    # set final weights
    fst.set_F(State(len(s)), semiring(-1.0))

    return fst

In [ ]:
# visualise the transducer
fst = string_fst("abc", semiring=Log)
fst

WFSA(4 states, <class '__main__.Log'>)

### Subquestion c) - Edit Distance Transducer

As we will not learn the weights on the arcs, you can set them to some pre-defined constants. You should choose the weights such that the pathsum that needs to be computed in e) does not diverge (this is caused by the weights of the self-loops in the transducer). 

In [ ]:
def edit_distance_fst(source_alphabet, target_alphabet, semiring=Log):
    """ Takes as input two sets of characters (source and target 
    alphabets) and a semiring and returns an edit-distance transducer
    """

    target = target_alphabet
    source = source_alphabet
    fst = FST(semiring)
    one = fst.R.one
    
    for j, t in enumerate(target):
        for i, s in enumerate(source):
          if i != j+1:
            fst.add_arc(State(i), ε, Sym(t), State(j+1), semiring(-20.0))
          for ii, ss in enumerate(source):
              if i != j+1:
                fst.add_arc(State(i), Sym(ss), Sym(t), State(j+1), semiring(-45.0))
        fst.set_F(State(j+1), semiring(-1/len(target)))
    i = 0
    for q in fst.Q:
      for i, s in enumerate(source):
        fst.add_arc(q, Sym(s), ε, q, semiring(-20.0))
      i = i + 1

    fst.set_I(State(0), semiring(-1.0))

    return fst

In [ ]:
fst = edit_distance_fst({'a', 'b', 'c'}, {'d', 'e'}, semiring=Log)
fst

WFSA(3 states, <class '__main__.Log'>)

### Subquestion d) - Composition

We provide some composition test cases for you to inspect.

In [ ]:
with open('test_cases.pkl', 'rb') as in_file:
    examples = dill.load(in_file)

In [ ]:
# FST 1
examples[1]['fst1']['fst']

WFSA(2 states, <class 'rayuela.base.semiring.Tropical'>)

In [ ]:
# FST 2
examples[1]['fst2']['fst']

WFSA(2 states, <class 'rayuela.base.semiring.Tropical'>)

In [ ]:
# composition
examples[1]['composition']['fst']

WFSA(4 states, <class 'rayuela.base.semiring.Tropical'>)

In [ ]:
# semiring
examples[3]['semiring']

rayuela.base.semiring.Real

In [ ]:
def compose(fst1, fst2):
    """ Takes as input two transducers and returns
    their composition
    """
    # The machines must be in the same semiring
    assert fst1.R == fst2.R

    comp_fst = FST(R=fst1.R)

    # Copy
    first = fst1.copy()
    second = fst2.copy() 

    last_state = []
    last_state_2 = []

    for el in first.I:
      first_state = el[0]
    for el in first.F:
      last_state.append(el[0])
    for el in second.I:
      first_state_2 = el[0]
    for el in second.F:
      last_state_2.append(el[0])

    # Augmentation
    for i, el in enumerate(fst1.Q):
      first.add_arc(el, ε, ε_1, el, first.R.one)
    for j, el2 in enumerate(fst2.Q):
      second.add_arc(el2, ε, ε_1, el2, second.R.one)

    # save initial and final weights
    weights = []
    weights2 = []

    for el in first.λ:
        weights.append(first.λ[el])
        break
    for el in first.ρ:
        weights.append(first.ρ[el])
        break
    for el2 in second.λ:
        weights2.append(second.λ[el2])
        break
    for el2 in second.ρ:
        weights2.append(second.ρ[el2])
        break

    # Composition init
    comp_fst = FST(R=fst1.R)

    arcs1 = []
    arcs2 = []
    for el in first.δ:
        for el2 in first.δ[el]:
            for el3 in first.δ[el][el2]:
                arcs1.append([el,el2,el3,first.δ[el][el2][el3]])
                    
    for el in second.δ:
        for el2 in second.δ[el]:
            for el3 in second.δ[el][el2]:
                arcs2.append([el,el2,el3,second.δ[el][el2][el3]])

    # correct augmentation
    loops = []
    loops2 = []
    for el in arcs1:
        if el[0] == el[2] and el[1] == (ε,ε):
            loops.append(el[0])
    for el in arcs2:
        if el[0] == el[2] and el[1] == (ε,ε):
            loops2.append(el[0])
    
    for i, el in enumerate(arcs1):
        if el[1][1] == ε_1:
            if el[0] not in loops:
                arcs1[i][1]=(el[1][0],ε)
            else:
              arcs1.remove(el)
    for i, el in enumerate(arcs2):
        if el[1][1] == ε_1:
            if el[0] not in loops2:
                arcs2[i][1]=(el[1][0],ε)
            else:
              arcs2.remove(el)

    # accessible composition
    stack = [(first_state,first_state_2)]
    visited = set()

    while len(stack)>0:
      q = stack.pop()
      for el in arcs1:
        if el[0] == q[0]:
          for el2 in arcs2:
            if el2[0] == q[1]:
              if el[1][1] == el2[1][0]:
                if el[1][0] == el[1][1] == el2[1][0] == el2[1][1] == ε and (el[0],el2[0]) == (el[2],el2[2]) : # delete the useless arcs
                    True
                else: # check if the state is ammissible
                    comp_fst.set_arc(PairState(el[0],el2[0]), el[1][0], el2[1][1], PairState(el[2],el2[2]), comp_fst.R.__mul__(el[3],el2[3]))
                    if (el[2],el2[2]) not in visited:
                      stack.append((el[2],el2[2]))
                      visited.add((el[2],el2[2]))             
    # add weights in first and last node
    comp_fst.set_I(PairState(first_state,first_state_2), comp_fst.R.__mul__(weights[0],weights2[0]))
    for i, el in enumerate(last_state):
      for j, el2 in enumerate(last_state_2):
        if PairState(el,el2) in comp_fst.Q:
          comp_fst.set_F(PairState(el,el2), comp_fst.R.__mul__(weights[1],weights2[1]))
    return comp_fst

In [ ]:
i=4
examples[i]['composition']['fst']

WFSA(4 states, <class 'rayuela.base.semiring.Boolean'>)

In [ ]:
compose(examples[i]['fst1']['fst'],examples[i]['fst2']['fst'])

WFSA(4 states, <class 'rayuela.base.semiring.Boolean'>)

### Subquestion e) - Lehmann's Algorithm

In [ ]:
class Pathsum:
    def __init__(self, fsa):

        # basic FSA stuff
        self.fsa = fsa
        self.R = fsa.R
        self.N = self.fsa.num_states

        # state dictionary
        self.I = {}
        for n, q in enumerate(self.fsa.Q):
            self.I[q] = n

        # lift into the semiring
        self.W, self.W_dict, self.alpha, self.beta = self.lift()

        
    def lift(self):
        """ creates the weight matrix, initial weight vector,
        final weight vector from the automaton """
        W = self.R.zeros((self.N, self.N))
        W_dict = {a: {} for a in self.fsa.Sigma}
        alpha = self.R.zeros((self.N,))
        beta = self.R.zeros((self.N,))
        for a in self.fsa.Sigma:
            for b in self.fsa.Delta:
                W_dict[a][b] = self.R.zeros((self.N, self.N))
        for p in self.fsa.Q:
            alpha[self.I[p]] += self.fsa.λ[p]
            beta[self.I[p]] += self.fsa.ρ[p]
            for (a, b), q, w in self.fsa.arcs(p):
                W[self.I[p], self.I[q]] += w
                W_dict[a][b][self.I[p], self.I[q]] = w
        return W, W_dict, alpha, beta
    
    def lehmann(self):
        """
        Lehmann's (1977) algorithm.
        """
        # TODO: implement
        W = self.W
        N = self.N
        R = W
        for k in range(N):
          for i in range(N):
            for j in range(N):
              R[i,j] = self.R.__add__(R[i,j],self.R.__mul__(self.R.__mul__(R[i,k],self.R.star(R[k,k])),R[k,j]))
        for i in range(N):
          for j in range(N):
            if i == j:
              R[i,j] = self.R.__add__(R[i,j],self.R.one)
        return R

    def _iterate(self, K):
        P = self.R.diag(self.N)
        for n in range(K):
          P += self.W @ P
        return P

    def _fixpoint(self, K=200):
        if self.fsa.R.idempotent:
          return self._iterate(self.fsa.num_states)

        diag = self.R.zeros((self.fsa.num_states, self.fsa.num_states))
        for n in range(self.fsa.num_states):
            diag[n, n] = self.fsa.R.one
        P_old = diag

        for _ in range(K):
            P_new = diag + self.W @ P_old
            P_old = P_new

        return P_old

    def fixpoint(self):

        P = self._fixpoint()
        W = {}

        for p in self.fsa.Q:
            for q in self.fsa.Q:
                if p in self.I and q in self.I:
                    W[p, q] = P[self.I[p], self.I[q]]
                elif p == q:
                    W[p, q] = self.R.one
                else:
                    W[p, q] = self.R.zero

        return frozendict(W)

In [ ]:
pathsum = Pathsum(examples[3]['fst1']['fst'])
print(pathsum.fixpoint())
print(pathsum.lehmann())

frozendict.frozendict({(0, 0): 1.204687055617909, (0, 1): 0.049703748026656, (0, 2): 0.238528037012346, (1, 0): 0.423848011191222, (1, 1): 1.064607810824038, (1, 2): 0.083921906215862, (2, 0): 0.297575363071423, (2, 1): 0.220654517754702, (2, 2): 1.058919921888142})
[[1.210030479232752 0.052036796373853 0.249169852930657]
 [0.442313809320239 1.071795077141898 0.118150643517558]
 [0.315208576189901 0.226634327641494 1.085202126621377]]


### Subquestion f) - Normalizer

We provide some test cases for you to inspect.

In [ ]:
k = 0
examples[k]['fst1']['Z'], examples[k]['fst2']['Z'], examples[k]['composition']['Z']

(0.00138425, 0.000997195294118, 7.07192371e-07)

In [ ]:
def normalizer(fst):
    """ Takes as input a finite-state transducer and
    returns its normalizer
    """
    pathsum = Pathsum(fst)
    alpha, beta = pathsum.alpha, pathsum.beta
    N = pathsum.N
    Z = pathsum.R.zero
    R = pathsum._fixpoint()
    for i in range(N):
      for k in range(N):
        Z = pathsum.R.__add__(Z,pathsum.R.__mul__(pathsum.R.__mul__(alpha[i],R[i,k]),beta[k]))
    return Z

### Subquestion g) - Log-likelihood

In [ ]:
def log_likelihood(source_alphabet, target_alphabet, source_str, target_str):
    """ Takes as input the source and target alphabets (sets of characters)
    and a pair of source and target strings and returns the log-likelihood
    """
    SF1 = string_fst(source_str)
    EDF = edit_distance_fst(source_alphabet, target_alphabet)
    C1 = compose(SF1,EDF)
    SF2 = string_fst(target_str)
    C2 = compose(C1,SF2)
    Z = normalizer(C2)
    
    return Z

### Subquestion h) - Decoding

In [ ]:
def to_semiring(fst, semiring):
    """ Lifts an WFST into a different semiring
    """
    oone, one = fst.R.one, semiring.one
    nfst = FST(semiring)

    # initial weights
    for q, w in fst.I:
        if w == oone:
            nfst.set_I(q, one)
        else:
            nfst.set_I(q, semiring(w.score))

    # final weights
    for q, w in fst.F:
        if w == oone:
            nfst.set_F(q, one)
        else:
            nfst.set_F(q, semiring(w.score))

    # arcs
    for p in fst.Q:
        for (a, b), q, w in fst.arcs(p):
            if w == oone:
                nfst.add_arc(p, a, b, q, one)
            else:
                nfst.add_arc(p, a, b, q, semiring(w.score))

    return nfst

In [ ]:
class Arctic(Semiring):
    def __init__(self, score):
        self.score = score

    def star(self):
        return self.one

    def __float__(self):
        return float(self.score)

    def __int__(self):
        return int(self.score)

    def __add__(self, other):
        return Arctic(max(self.score, other.score))

    def __mul__(self, other):
        if other is self.one:
            return self
        if self is self.one:
            return other
        if other is self.zero:
            return self.zero
        if self is self.zero:
            return self.zero
        return Arctic(self.score + other.score)

    def __invert__(self):
        return Arctic(-self.score)

    def __truediv__(self, other):
        return Arctic(self.score - other.score)

    def __lt__(self, other):
        return self.score < other.score

    def __repr__(self):
        return f"Arctic({self.score})"

    def __str__(self):
        return str(self.score)


Arctic.zero = Arctic(-float("inf"))
Arctic.one = Arctic(0.0)
Arctic.idempotent = True
Arctic.inferior = True
Arctic.cancellative = True

In [ ]:
def decode(in_str, edit_distance_fst):
    """ Gets as input a source string and an
    edit-distance transducer and returns the weight of the most
    probable target string
    """
    SF1 = string_fst(in_str)
    C = compose(SF1,edit_distance_fst)
    lifted = to_semiring(C, semiring = Arctic)
    return normalizer(lifted)